<a href="https://colab.research.google.com/github/Ahmed230460/News-Summarization-and-Search-Application/blob/main/news_summarization_with_langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain openai requests

In [10]:
!pip install langchain langchain-openai groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 14.6 MB/s eta 0:00:00


In [3]:
!pip install langchain_groq langchain_community --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.2/122.2 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.2 MB/s eta 0:00:00


In [15]:
!pip install faiss-cpu chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 46.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 47.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.3/79.3 kB 8.0 MB/s eta 0:00

In [33]:
import os
import json
import requests
from langchain_groq import ChatGroq
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.summarize import load_summarize_chain
from langchain.docstore.document import Document

In [34]:
# Set API Keys
NEWS_API_KEY = "134f287e31a54aac9d3de1790b50f2da"
GROQ_API_KEY = "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"
os.environ["GROQ_API_KEY"] = GROQ_API_KEY

In [35]:
class NewsRetriever:
    def __init__(self, api_key):
        self.api_key = api_key
        self.base_url = "https://newsapi.org/v2/everything"

    def fetch_news(self, topic, max_results=3):
        """Fetch news articles for a given topic."""
        params = {"q": topic, "apiKey": self.api_key, "pageSize": max_results}
        response = requests.get(self.base_url, params=params).json()
        return response.get("articles", [])

In [47]:
from langchain.embeddings import HuggingFaceEmbeddings

class EmbeddingEngine:
    def __init__(self):
        self.embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")  # Local model
        self.vector_db = Chroma(collection_name="news_embeddings", embedding_function=self.embedding_model)
        self.text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)

    def add_article(self, title, content):
        """Embed and store an article."""
        docs = [Document(page_content=text) for text in self.text_splitter.split_text(content)]
        self.vector_db.add_documents(docs)

    def search_similar_articles(self, query, top_k=3):
        """Search for similar articles using vector search."""
        results = self.vector_db.similarity_search(query, k=top_k)
        return [res.page_content for res in results]


In [48]:
class Summarizer:
    def __init__(self):
        self.llm = ChatGroq(model_name="llama3-8b-8192")

    def summarize_basic(self, text):
        """Basic summarization using a simple prompt."""
        prompt = PromptTemplate.from_template("Summarize the following text briefly:\n\n{text}\n\nSummary:")
        chain = prompt | self.llm | StrOutputParser()
        return chain.invoke({"text": text})

    def summarize_advanced(self, text):
        """Advanced summarization using LangChain's RefineChain."""
        docs = [Document(page_content=text)]
        chain = load_summarize_chain(self.llm, chain_type="refine")
        return chain.run(docs)

In [44]:
class UserPreferences:
    def __init__(self, file_path="user_preferences.json"):
        self.file_path = file_path
        self.data = {"favorite_topics": [], "search_history": []}
        self._load_preferences()

    def _load_preferences(self):
        """Load user preferences from JSON file."""
        if os.path.exists(self.file_path):
            with open(self.file_path, "r") as file:
                self.data = json.load(file)

    def save_preferences(self):
        """Save user preferences to JSON file."""
        with open(self.file_path, "w") as file:
            json.dump(self.data, file, indent=4)

    def add_favorite_topic(self, topic):
        """Add a topic to favorites."""
        if topic not in self.data["favorite_topics"]:
            self.data["favorite_topics"].append(topic)
            self.save_preferences()

    def add_search_history(self, topic, brief_summary, detailed_summary):
        """Store search history with summaries (limit to last 10)."""
        self.data["search_history"].append({
            "topic": topic,
            "brief_summary": brief_summary,
            "detailed_summary": detailed_summary
        })
        self.data["search_history"] = self.data["search_history"][-10:]  # Keep only the last 10 searches
        self.save_preferences()

    def get_favorite_topics(self):
        """Retrieve favorite topics."""
        return self.data["favorite_topics"]

    def get_search_history(self):
        """Retrieve search history."""
        return self.data["search_history"]

    def clear_search_history(self):
        """Clear search history."""
        self.data["search_history"] = []
        self.save_preferences()

In [49]:
if __name__ == "__main__":
    news_retriever = NewsRetriever(NEWS_API_KEY)
    summarizer = Summarizer()
    embedding_engine = EmbeddingEngine()
    user_prefs = UserPreferences()

    while True:
        print("\nMENU")
        print("1 - Search for news")
        print("2 - View search history")
        print("3 - View favorite topics")
        print("4 - Clear search history")
        print("5 - Exit")
        choice = input("Select an option: ").strip()

        if choice == "1":
            topic = input("Enter a topic: ").strip()
            num_articles = int(input("How many articles do you want to retrieve? (1-5): ").strip())

            articles = news_retriever.fetch_news(topic, max_results=num_articles)
            if not articles:
                print("No articles found for this topic.")
                continue

            for i, article in enumerate(articles, 1):
                title = article.get("title", "No title available")
                content = article.get("content", "No content available")

                brief_summary = summarizer.summarize_basic(content)
                detailed_summary = summarizer.summarize_advanced(content)

                print(f"\nArticle {i}: {title}")
                print(f"Brief Summary: {brief_summary}")
                print(f"Detailed Summary: {detailed_summary[:300]}...")  # Show first 300 chars for readability

                # Store in vector DB
                embedding_engine.add_article(title, content)

                # Store in search history
                user_prefs.add_search_history(topic, brief_summary, detailed_summary)

            fav = input("Do you want to add this topic to favorites? (yes/no): ").strip().lower()
            if fav == "yes":
                user_prefs.add_favorite_topic(topic)

        elif choice == "2":
            history = user_prefs.get_search_history()
            if history:
                print("\nSearch History:")
                for idx, entry in enumerate(history, 1):
                    print(f"\n{idx}. Topic: {entry['topic']}")
                    print(f"Brief Summary: {entry['brief_summary']}")
                    print(f"Detailed Summary: {entry['detailed_summary'][:200]}...")
            else:
                print("\nNo search history found.")

        elif choice == "3":
            favorites = user_prefs.get_favorite_topics()
            print("\nFavorite Topics:", favorites if favorites else "No favorite topics yet.")

        elif choice == "4":
            confirm = input("Are you sure you want to clear search history? (yes/no): ").strip().lower()
            if confirm == "yes":
                user_prefs.clear_search_history()
                print("Search history cleared.")

        elif choice == "5":
            print("Exiting...")
            break

        else:
            print("Invalid choice! Please select again.")


MENU
1 - Search for news
2 - View search history
3 - View favorite topics
4 - Clear search history
5 - Exit
Select an option: 1
Enter a topic: AI
How many articles do you want to retrieve? (1-5): 1

Article 1: T-Mobile’s parent company is making an ‘AI Phone’ with Perplexity Assistant
Brief Summary: T-Mobile's Magenta AI push will offer AI apps, including Perplexity, for existing smartphones.
Detailed Summary: T-Mobile's Magenta AI push will offer AI apps, including Perplexity, for existing smartphones....
Do you want to add this topic to favorites? (yes/no): yes

MENU
1 - Search for news
2 - View search history
3 - View favorite topics
4 - Clear search history
5 - Exit
Select an option: 1
Enter a topic: cyber security
How many articles do you want to retrieve? (1-5): 3

Article 1: US Government's Top Election Watchdog Forced to Stop Election Security Work
Brief Summary: The Cybersecurity and Infrastructure Security Agency has halted all election security work and is conducting a revi